Fundamentos de ciência de dados - Luiz Davi e Thiago Cavalcanti

In [1]:
from nba_api_utils import *
from nba_api.stats.endpoints import teamgamelog, boxscoretraditionalv2
import pandas as pd

# Obtendo informações do time

In [2]:
nome_do_time = 'Denver Nuggets'
periodo_anterior = '2023-24'
periodo_atual = '2024-25'

# RF7 – Apresentar os jogos do seu time [temporada 23-24 e temporada atual], conforme Tabela 6.

In [25]:
def obter_jogos_do_time_na_temporada(nome_do_time, periodo):
    time = obter_time(nome_do_time)
    time_id = time['id']
    
    jogos = teamgamelog.TeamGameLog(team_id=time_id, season=periodo)
    dados_jogos = jogos.get_data_frames()[0]
    dados_jogos = dados_jogos.head(5)

    jogos_formatados = []

    for indice, linha in dados_jogos.iterrows():
        data_jogo = linha['GAME_DATE']
        adversario = linha['MATCHUP'][-3:].strip()
        if linha['WL'] != None:
            resultado = 'Vitória' if 'W' in linha['WL'] else 'Derrota'
        else:
            resultado = 'Indefinido'
        casa_ou_fora = 'Casa' if '@' not in adversario else 'Fora'

        game_id = linha['Game_ID']
        jogo_boxscore = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id)
        dados_boxscore = jogo_boxscore.get_data_frames()[0]

        if casa_ou_fora == 'Casa':
            placar_time = linha['PTS']
            placar_adversario = dados_boxscore[dados_boxscore['TEAM_ID'] != time_id]['PTS'].sum()
        else:
            placar_adversario = linha['PTS']
            placar_time = dados_boxscore[dados_boxscore['TEAM_ID'] != time_id]['PTS'].sum()
        
        placar = f'{int(placar_time)} - {int(placar_adversario)}'

        jogos_formatados.append([data_jogo, adversario, resultado, casa_ou_fora, placar])

    tabela_jogos = pd.DataFrame(jogos_formatados, columns=['Data do jogo', 'Adversário', 'Vitória ou derrota', 'Casa ou fora', 'Placar'])
    return tabela_jogos

## Estatísticas da temporada anterior

In [26]:
estatisticas_passadas = obter_jogos_do_time_na_temporada(nome_do_time, periodo_anterior)
estatisticas_passadas

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

## Estatísticas da temporada atual

In [16]:
estatisticas_atuais = obter_jogos_do_time_na_temporada(nome_do_time, periodo_atual)
estatisticas_atuais

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)